# 馬のデータ分析

In [41]:
# 必要なパッケージのインポート
import pandas as pd
import pandas_profiling as pdp

In [42]:
# データの読み込み
df = pd.read_csv('horse_data.csv')

#### columnsの名前が日本語のせいで文字化けするため英語に変換する

In [43]:
# columns名の確認
df.columns

Index(['日付', '開催', '天気', 'R', 'レース名', '映像', '頭数', '枠番', '馬番', 'オッズ', '人気',
       '着順', '騎手', '斤量', '距離', '馬場', '馬場指数', 'タイム', '着差', 'ﾀｲﾑ指数', '通過', 'ペース',
       '上り', '馬体重', '厩舎ｺﾒﾝﾄ', '備考', '勝ち馬(2着馬)', '賞金', 'horse_id', 'race_id',
       'jockey_id'],
      dtype='object')

In [44]:
df = df.rename(columns={'日付':'day','開催':'hold','天気':'weather', 'R':'round', 
    'レース名':'race_name', '映像':'movie', '頭数':'starters_num', '枠番':'flame', '馬番':'horse_num', 'オッズ':'odds',
    '人気':'popularity','着順':'arrival', '騎手':'jockey', '斤量':'weight', '距離':'meter', '馬場':'state',
    '馬場指数':'state_value', 'タイム':'time', '着差':'arrival_diff', 'ﾀｲﾑ指数':'time_value', '通過':'passing', 'ペース':'pace',
    '上り':'3f', '馬体重':'horse_weight', '厩舎ｺﾒﾝﾄ':'comment', '備考':'other', '勝ち馬(2着馬)':'second', '賞金':'winning'})

In [45]:
# 列名が変わったことの確認
df.columns

Index(['day', 'hold', 'weather', 'round', 'race_name', 'movie', 'starters_num',
       'flame', 'horse_num', 'odds', 'popularity', 'arrival', 'jockey',
       'weight', 'meter', 'state', 'state_value', 'time', 'arrival_diff',
       'time_value', 'passing', 'pace', '3f', 'horse_weight', 'comment',
       'other', 'second', 'winning', 'horse_id', 'race_id', 'jockey_id'],
      dtype='object')

### 余計な行、列を削除

##### データがない列を削除

In [46]:
df.drop(['movie', 'comment', 'other', 'time_value', 'state_value'], axis=1, inplace=True)

##### 地方競馬場のデータを削除

In [48]:
# 地方競馬場はラウンド表記がないため数字の正規表現で地方競馬場の結果は省ける
df = df[df['hold'].str.match(r'[0-9]+', na=False)]

In [49]:
# データの確認
df.head(5)

,day,hold,weather,round,race_name,starters_num,flame,horse_num,odds,popularity,...,arrival_diff,passing,pace,3f,horse_weight,second,winning,horse_id,race_id,jockey_id
5,2021/01/17,1小倉2,晴,3.0,4歳以上1勝クラス,14.0,2.0,2,156.4,14.0,...,1.3,10-10,34.3-36.4,36.7,458(+6),クーファピーカブー,NaN,2017101464,202110010203,1143.0
6,2020/11/08,3福島4,晴,7.0,3歳以上1勝クラス,13.0,8.0,12,38.6,9.0,...,1.5,9-11,32.2-37.0,37.7,452(+4),サノマナ,NaN,2017101464,202003020512,1119.0
7,2020/07/18,2福島5,曇,12.0,3歳以上1勝クラス,15.0,3.0,6,30.3,10.0,...,1.2,7-7,30.9-37.1,37.4,448(0),タイキメサイア,NaN,2017101464,202003020512,1119.0
8,2020/05/02,1福島7,晴,10.0,しゃくなげ賞(1勝クラス),16.0,8.0,16,41.7,14.0,...,1.5,8-13,34.0-35.5,36.4,448(-10),ベッラヴォルタ,NaN,2017101464,202003010710,1143.0
9,2020/03/07,2中山3,曇,6.0,3歳1勝クラス,16.0,7.0,13,14.6,5.0,...,1.1,3-3,33.4-38.3,39.1,458(-2),ヴァンドゥメール,NaN,2017101464,202006020306,1119.0


##### １８頭以上の馬がいるもしくは１頭もいないレースの行を削除

In [50]:
df = df[(df['starters_num'] <= 18) & (df['starters_num'] >= 4)]

In [51]:
# データの確認
df.head(5)

,day,hold,weather,round,race_name,starters_num,flame,horse_num,odds,popularity,...,arrival_diff,passing,pace,3f,horse_weight,second,winning,horse_id,race_id,jockey_id
5,2021/01/17,1小倉2,晴,3.0,4歳以上1勝クラス,14.0,2.0,2,156.4,14.0,...,1.3,10-10,34.3-36.4,36.7,458(+6),クーファピーカブー,NaN,2017101464,202110010203,1143.0
6,2020/11/08,3福島4,晴,7.0,3歳以上1勝クラス,13.0,8.0,12,38.6,9.0,...,1.5,9-11,32.2-37.0,37.7,452(+4),サノマナ,NaN,2017101464,202003020512,1119.0
7,2020/07/18,2福島5,曇,12.0,3歳以上1勝クラス,15.0,3.0,6,30.3,10.0,...,1.2,7-7,30.9-37.1,37.4,448(0),タイキメサイア,NaN,2017101464,202003020512,1119.0
8,2020/05/02,1福島7,晴,10.0,しゃくなげ賞(1勝クラス),16.0,8.0,16,41.7,14.0,...,1.5,8-13,34.0-35.5,36.4,448(-10),ベッラヴォルタ,NaN,2017101464,202003010710,1143.0
9,2020/03/07,2中山3,曇,6.0,3歳1勝クラス,16.0,7.0,13,14.6,5.0,...,1.1,3-3,33.4-38.3,39.1,458(-2),ヴァンドゥメール,NaN,2017101464,202006020306,1119.0


##### 出走しなかったレースの情報を削除

In [55]:
df = df[df['arrival'].str.match(r'[0-9]+', na=False)]

### NAN値の対処

In [56]:
# nan値のある列の確認
df.isnull().any()

day             False
hold            False
weather         False
round           False
race_name       False
starters_num    False
flame           False
horse_num       False
odds            False
popularity      False
arrival         False
jockey          False
weight          False
meter           False
state           False
time            False
arrival_diff    False
passing         False
pace            False
3f              False
horse_weight    False
second          False
winning          True
horse_id        False
race_id         False
jockey_id       False
dtype: bool

##### 獲得賞金(winning)のnan値の処理

In [57]:
# nan値は賞金を獲得できていないため0でfillする
df.fillna({'winning':0}, inplace=True)

In [58]:
# nan値がなくなったかの確認
df.isnull().any()

day             False
hold            False
weather         False
round           False
race_name       False
starters_num    False
flame           False
horse_num       False
odds            False
popularity      False
arrival         False
jockey          False
weight          False
meter           False
state           False
time            False
arrival_diff    False
passing         False
pace            False
3f              False
horse_weight    False
second          False
winning         False
horse_id        False
race_id         False
jockey_id       False
dtype: bool

### 型を正しいものに修正(数値がstrになっているデータなど)

In [66]:
# 型の確認
df.dtypes

day              object
hold             object
weather          object
round           float64
race_name        object
starters_num    float64
flame           float64
horse_num         int64
odds            float64
popularity      float64
arrival           int64
jockey           object
weight          float64
meter            object
state            object
time             object
arrival_diff    float64
passing          object
pace             object
3f              float64
horse_weight     object
second           object
winning         float64
horse_id          int64
race_id           int64
jockey_id       float64
dtype: object

##### レース距離(meter)がobject型になっているので修正

In [68]:
# まずは中身を確認
df['meter'].head(5)

5    ダ1000
6    ダ1150
7    ダ1150
8    芝1200
9    ダ1200
Name: meter, dtype: object

In [69]:
# レースの馬場(芝、ダート)はレース側で持ってこれるデータなので削除する
df['meter'] = df['meter'].apply(lambda x:int(re.search(r'[0-9]+', x).group()))

In [70]:
# 変わったことの確認
df['meter'].head(5)

5    1000
6    1150
7    1150
8    1200
9    1200
Name: meter, dtype: int64

### 順位と他の値の相関を見るため順位を数字の値に統一する

In [59]:
# 必要なパッケージのインポート
import re
import numpy as np

In [60]:
# あとで使い回すために関数形にする
def ranking_category2num(ranking):
    ranking = str(ranking)
    return int(re.search(r'[0-9]+', ranking).group())

In [61]:
# 数字に変換
df['arrival'] = df['arrival'].apply(lambda x: ranking_category2num(x))

In [62]:
# 内容の確認
df.head(5)

,day,hold,weather,round,race_name,starters_num,flame,horse_num,odds,popularity,...,arrival_diff,passing,pace,3f,horse_weight,second,winning,horse_id,race_id,jockey_id
5,2021/01/17,1小倉2,晴,3.0,4歳以上1勝クラス,14.0,2.0,2,156.4,14.0,...,1.3,10-10,34.3-36.4,36.7,458(+6),クーファピーカブー,0.0,2017101464,202110010203,1143.0
6,2020/11/08,3福島4,晴,7.0,3歳以上1勝クラス,13.0,8.0,12,38.6,9.0,...,1.5,9-11,32.2-37.0,37.7,452(+4),サノマナ,0.0,2017101464,202003020512,1119.0
7,2020/07/18,2福島5,曇,12.0,3歳以上1勝クラス,15.0,3.0,6,30.3,10.0,...,1.2,7-7,30.9-37.1,37.4,448(0),タイキメサイア,0.0,2017101464,202003020512,1119.0
8,2020/05/02,1福島7,晴,10.0,しゃくなげ賞(1勝クラス),16.0,8.0,16,41.7,14.0,...,1.5,8-13,34.0-35.5,36.4,448(-10),ベッラヴォルタ,0.0,2017101464,202003010710,1143.0
9,2020/03/07,2中山3,曇,6.0,3歳1勝クラス,16.0,7.0,13,14.6,5.0,...,1.1,3-3,33.4-38.3,39.1,458(-2),ヴァンドゥメール,0.0,2017101464,202006020306,1119.0


### 距離適性を数値として求める

In [76]:
# 移行した時のことを考えて関数化
# def distance_aptitude(df):
#     for horse_id in df['horse_id'].unique():
#         horse_df = df[df['horse_id'] == horse_id]

#         # データ量が2以下(出場レースが2以下)の場合は一律で1600mとする(データ量が少ないため出せないor出せても信頼性に欠けるため)
#         if len(horse_df) <= 2:
#             meter_apt = 1600
#             horse_df['meter_apt'] = meter_apt
#             continue

#         distances = []
#         weights = []

#         for row in horse_df.itertuples():
#             w = abs((int(row.starters_num) - int(row.arrival) + 1)/int(row.starters_num))

#             distances.append(row.meter)
#             weights.append(w)
        
#         meter_apt = np.average(distances, weights=weights)

#         df.loc[df['horse_id'] == horse_id, 'meter_apt'] = meter_apt

#     return df

In [85]:
for horse_id in df['horse_id'].unique():
    horse_df = df[df['horse_id'] == horse_id]

    # データ量が2以下(出場レースが2以下)の場合は一律で1600mとする(データ量が少ないため出せないor出せても信頼性に欠けるため)
    if len(horse_df) <= 2:
        meter_apt = 1600
        df.loc[df['horse_id'] == horse_id, 'meter_apt'] = meter_apt
        continue

    distances = []
    weights = []

    for row in horse_df.itertuples():
        w = abs((int(row.starters_num) - int(row.arrival) + 1)/int(row.starters_num))

        distances.append(row.meter)
        weights.append(w)
    
    meter_apt = np.average(distances, weights=weights)

    df.loc[df['horse_id'] == horse_id, 'meter_apt'] = meter_apt


In [87]:
# うまくいったか確認
df.head(10)

,day,hold,weather,round,race_name,starters_num,flame,horse_num,odds,popularity,...,passing,pace,3f,horse_weight,second,winning,horse_id,race_id,jockey_id,meter_apt
5,2021/01/17,1小倉2,晴,3.0,4歳以上1勝クラス,14.0,2.0,2,156.4,14.0,...,10-10,34.3-36.4,36.7,458(+6),クーファピーカブー,0.0,2017101464,202110010203,1143.0,1199.407089
6,2020/11/08,3福島4,晴,7.0,3歳以上1勝クラス,13.0,8.0,12,38.6,9.0,...,9-11,32.2-37.0,37.7,452(+4),サノマナ,0.0,2017101464,202003020512,1119.0,1199.407089
7,2020/07/18,2福島5,曇,12.0,3歳以上1勝クラス,15.0,3.0,6,30.3,10.0,...,7-7,30.9-37.1,37.4,448(0),タイキメサイア,0.0,2017101464,202003020512,1119.0,1199.407089
8,2020/05/02,1福島7,晴,10.0,しゃくなげ賞(1勝クラス),16.0,8.0,16,41.7,14.0,...,8-13,34.0-35.5,36.4,448(-10),ベッラヴォルタ,0.0,2017101464,202003010710,1143.0,1199.407089
9,2020/03/07,2中山3,曇,6.0,3歳1勝クラス,16.0,7.0,13,14.6,5.0,...,3-3,33.4-38.3,39.1,458(-2),ヴァンドゥメール,0.0,2017101464,202006020306,1119.0,1199.407089
10,2020/01/05,1中山1,晴,1.0,3歳未勝利,16.0,4.0,7,3.6,2.0,...,5-5,33.9-39.4,38.9,460(+2),(グルアーブ),510.0,2017101464,202006010101,1119.0,1199.407089
11,2019/10/20,4東京7,晴,3.0,2歳新馬,16.0,7.0,14,8.3,4.0,...,3-3,30.3-36.0,35.9,458(0),メイショウテンスイ,280.0,2017101464,201905040703,1119.0,1199.407089
12,2022/01/09,1中山3,晴,10.0,若潮S(3勝クラス),13.0,7.0,10,9.0,5.0,...,11-9-10,35.0-35.3,34.9,442(+2),エイシンチラー,0.0,2017103291,202206010310,1150.0,1409.102194
13,2021/12/12,5中山4,晴,12.0,3歳以上2勝クラス,13.0,6.0,8,22.0,8.0,...,2-2-2,35.8-35.1,34.6,440(0),(ワザモノ),1100.0,2017103291,202006050612,1150.0,1409.102194
14,2021/11/21,5東京6,曇,12.0,3歳以上2勝クラス,18.0,8.0,18,51.6,12.0,...,11-10,35.3-34.7,34.1,440(+22),ミスビアンカ,0.0,2017103291,202006050612,1150.0,1409.102194


### 脚質を数値として求める

In [ ]:
def running_style(df):
    
    for horse_id in df['horse_id'].unique():
        horse_df = df[df['horse_id'] == horse_id]

        race = []
        weight = []
        for row in horse_df.itertuples():
            passing_result = [int(x) for x in row.passing.split('-')]
            mean_val = sum(passing_result)/len(passing_result)
            w = abs((int(row.starters_num) - int(row.arrival) + 1)/int(row.starters_num))
            race.append(mean_val)
            weight.append(w)
        
        # 馬の情報が2レース以下の場合適当に6の値とする
        if len(horse_df) <= 2:
            df.loc[df['horse_id'] == horse_id, 'run_style'] = 6
        else:
            df.loc[df['horse_id'] == horse_id, 'run_style'] = np.average(race, weights=weight)
        
    return df

### レース開催時の獲得賞金を数値として求める

## データの分析

In [ ]:
pdp.ProfileReport(df)